In [1]:
import os
import re
import xlwt
import xlrd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib.ticker import PercentFormatter
import collections
import jieba
import jieba.analyse as analyse
import json
import seaborn as sns
from openpyxl import load_workbook
from operator import itemgetter
from wordcloud import WordCloud
from imageio import imread

In [2]:
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 20)

save_path = '/Users/priscilla/Desktop/python/Fuguo/xueqiu_result'
os.chdir(save_path) 

In [3]:
# article
# read_path = '/Users/priscilla/Desktop/python/Fuguo/xueqiu_raw/article_data'
def csv_file_name(read_path):
    csv_file_name = []
    for root, dirs, files in os.walk(read_path):
        for file in files:
            if os.path.splitext(file)[1] == '.csv':
                csv_file_name.append(os.path.join(root, file))
    return csv_file_name

# csv_file_names = csv_file_name(read_path)

In [4]:
# merge article related csv into a single dataframe
def merging_csv(csv_list):
    frame = []

    for i in range(0,len(csv_list)):
        try: 
            df = pd.read_csv(csv_list[i], encoding = 'gb18030')
            frame.append(df)
        except:
            df = pd.read_csv(csv_list[i])
            frame.append(df)
    result = pd.concat(frame,sort=True) 
    return result

# article=merging_csv(csv_file_names)
# article=article[['user_id','view_count','title']]

In [55]:
# article
article=pd.read_excel('/Users/priscilla/Desktop/python/Fuguo/xueqiu_result/article.xlsx',index_col=0)

In [56]:
# deal with article related info
article['count_article']=1
article=article.sort_values(by='view_count',ascending=False)
grouped=article.groupby('user_id')

In [58]:
def best_article(x):
    return(x.iloc[0])
def best_four(x):
    return(','.join(x.iloc[0:4]))
article['best_4']=article['title']
article=grouped.agg({'view_count':np.mean,'count_article':np.sum,'title':best_article,'best_4':best_four})


In [62]:
article.loc[8417755168,'best_4']

'连接投资的一切——指数投资的奥秘,操作更新：如何将盈透账户绑定到雪盈证券,记一次愉快的美股转户操作（Firstrade为例）,「雪球招聘」新媒体运营，欢迎加入！'

In [11]:
# stock
save_path = '/Users/priscilla/Desktop/python/Fuguo/xueqiu_result'
watch_asset_final=pd.read_excel(save_path+'/'+'watch_asset_final.xlsx')
watch_asset_final=watch_asset_final.drop(["Unnamed: 0","Unnamed: 0.1"],axis=1)


In [12]:
# stock related info: region(A,H,US), # stock, indu
def region_per(x):
    n=len(x)
    dic={'A':sum(x=='A')/n,'H':sum(x=='H')/n,'US':sum(x=='US')/n}
    return dic

ls_US=['美国股票','美国基金']
ls_A=['A股主板股票','A股基金','A股股转债','A股企业和国家债券','质押回购债券','A股三板股票','A股科创板股票']
ls_H=['H股股票','H股股权','H股基金']

def make_simple(x):    
    if x in ls_US:
        return 'US'
    elif x in ls_A:
        return 'A'
    elif x in ls_H:
        return 'H'
    return '/'

watch_asset_final['region']=watch_asset_final['type_chinese'].apply(make_simple)
watch_asset_final['count_stock']=1
grouped = watch_asset_final.groupby(by='uid')

# number of stock & region
stock_group = grouped.agg({'count_stock':np.sum,'region':region_per})
stock_group['A']=stock_group['region'].apply(lambda x:x['A'])
stock_group['H']=stock_group['region'].apply(lambda x:x['H'])
stock_group['US']=stock_group['region'].apply(lambda x:x['US'])


In [49]:
# #重新爬取了每个用户的portfolio，将其合成一个portfolio列表用于后续portfolio每日数据爬取
# csv_file_name=csv_file_name('/Users/priscilla/Desktop/python/Fuguo/xueqiu_raw/xueqiu')

# #选取文件列表中的portfolio文件的目录存入列表portfolio
# def portfolio(csv_file_names):
#     portfolio = []
#     for file in csv_file_names:
#         if os.path.basename(file) == 'portfolio.csv':
#             portfolio.append(file)
#     return portfolio     
# portfolio_list=portfolio(csv_file_name)
# portfolio_raw = merging_csv(portfolio_list)
# portfolio_raw.to_excel('/Users/priscilla/Desktop/python/Fuguo/xueqiu_result/portfolio_list.xlsx')

In [13]:
# portfolio

# read csv
read_path = '/Users/priscilla/Desktop/python/Fuguo/xueqiu_raw/portfolio_new'
csv_file_names = csv_file_name(read_path)

# merge csv
#portfolio=merging_csv(csv_file_names)


In [14]:
# merge article related csv into a single dataframe
def merging_csv_withName(csv_list):
    frame = []

    for i in range(0,len(csv_list)):
        try: 
            df = pd.read_csv(csv_list[i], encoding = 'gb18030')
            df['portfolio_name']=re.search('portfolio_new/(.*)/portfolio_net_value\.csv',csv_list[i]).group(1)
            frame.append(df)
            
        except:
            df = pd.read_csv(csv_list[i])
            df['portfolio_name']=re.search('portfolio_new/(.*)/portfolio_net_value\.csv',csv_list[i]).group(1)
            frame.append(df)
    result = pd.concat(frame,sort=True) 
    return result


#get portfolio_net_value.csv
def portfolio_net_value(csv_file_names):
    portfolio_net_value= []
    for file in csv_file_names:
        if os.path.basename(file) == 'portfolio_net_value.csv':
            portfolio_net_value.append(file)
    return portfolio_net_value

In [15]:
port_temp_names=portfolio_net_value(csv_file_names)
port_temp=merging_csv_withName(port_temp_names)

In [16]:
import time,datetime
def get_time(x):
    x=x//1000
    dateArray=datetime.datetime.fromtimestamp(x)
    #time=dateArray.strftime("%Y/%m/%d %H:%M:%S")
    return dateArray

In [17]:
# check whether the portfolio has existed for 3y&1y
port_temp['date_time']=port_temp['time'].apply(get_time)
grouped=port_temp.groupby('portfolio_name')
date_range=grouped['date_time'].apply(lambda x: x.iloc[-1]-x[0]).to_frame()
date_range['3years']=(date_range['date_time']>datetime.timedelta(days=365*3+1))
date_range['1year']=(date_range['date_time']>datetime.timedelta(days=365))
date_range['exist_period']=date_range['date_time']
date_range['exist_period']=date_range['date_time'].astype('timedelta64[D]').astype(int) 
date_range['exist_period']=date_range['exist_period'].apply(lambda x:x/365)

In [18]:
# calculate the 3 years and 1 year annual return
temp=port_temp.merge(date_range,left_on='portfolio_name',right_index=True,how='left')
temp['value_3y_before']=temp.groupby("portfolio_name")['value'].shift(250*3)
temp['value_1y_before']=temp.groupby("portfolio_name")['value'].shift(250)

temp['3y_return']=(temp['value']-temp['value_3y_before'])/3
temp['1y_return']=(temp['value']-temp['value_1y_before'])

temp=temp.groupby('portfolio_name').apply(lambda x:x.iloc[-1,])
temp=temp[['exist_period','3years','1year','3y_return','1y_return']]

In [19]:
temp

,exist_period,3years,1year,3y_return,1y_return
portfolio_name,,,,,
SP1000000,0.000000,False,False,NaN,NaN
SP1000003,0.000000,False,False,NaN,NaN
ZH000102,2.090411,False,True,NaN,-0.1150
ZH000117,2.087671,False,True,NaN,0.1463
ZH000118,5.241096,True,True,0.130333,0.3245
...,...,...,...,...,...
ZH996304,3.082192,True,True,0.150833,0.6334
ZH996968,3.076712,True,True,0.279467,0.4380
ZH997464,0.043836,False,False,NaN,NaN


In [27]:
portfolio=pd.read_excel(save_path+'/'+'portfolio_list.xlsx',index_col=0)
portfolio.set_index('portfolio_name',inplace=True)


In [34]:
# portfolio
portfolio=pd.read_excel(save_path+'/'+'portfolio_list.xlsx',index_col=0)
portfolio.set_index('portfolio_name',inplace=True)
portfolio=portfolio.merge(temp,left_index=True,right_index=True,how='inner')


In [37]:
port=portfolio[['uid','name','annualized_gain','monthly_gain','exist_period','3years','1year','3y_return','1y_return']]
port['count_portfolio']=1
grouped=port.groupby('uid')
port=grouped.agg({'monthly_gain':np.mean,'count_portfolio':np.sum,'exist_period':np.mean,'annualized_gain':np.mean})


/Users/priscilla/Desktop/python/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [39]:
def first_name(x):
    return(x[0])

#1 year annualized_gain
portfolio=portfolio.sort_values(by='annualized_gain',ascending=False)
grouped=portfolio[portfolio['1year']].groupby('uid')
port['1y_return']=grouped['1y_return'].mean()
port['1y_return']=port['1y_return'].apply(lambda x:x*100)
port['best_port']=grouped['name'].apply(first_name)

#3 years annualized gain
grouped=portfolio.groupby('uid')
port['3y_return']=grouped['3y_return'].mean()
port['3y_return']=port['3y_return'].apply(lambda x:x*100)

portfolio=port

In [41]:
# industry
indu=pd.read_excel(save_path+'/'+'industry_g.xlsx',index_col=0)
indu=indu[['uid','INDUSTRY_GICS']]
indu.set_index('uid',inplace=True)

In [42]:
# user
user=pd.read_excel("/Users/priscilla/Desktop/python/Fuguo/xueqiu_result/user_1.xlsx")
user=user[['id','screen_name','verified_description','description','followers_count','friends_count','个人身份认证','自媒体身份认证','达人认证','公司认证','实名认证','实盘认证','雪球官方账号认证']]
user.set_index('id',inplace=True)

In [43]:
user['认证']=[[]]*len(user)
user['认证'][user['个人身份认证']==True]=user['认证'][user['个人身份认证']==True].apply(lambda x:x+['个人身份认证'])
user['认证'][user['自媒体身份认证']==True]=user['认证'][user['自媒体身份认证']==True].apply(lambda x:x+['自媒体身份认证'])
user['认证'][user['达人认证']==True]=user['认证'][user['达人认证']==True].apply(lambda x:x+['达人认证'])
user['认证'][user['公司认证']==True]=user['认证'][user['公司认证']==True].apply(lambda x:x+['公司认证'])
user['认证'][user['实名认证']==True]=user['认证'][user['实名认证']==True].apply(lambda x:x+['实名认证'])
user['认证'][user['实盘认证']==True]=user['认证'][user['实盘认证']==True].apply(lambda x:x+['实盘认证'])
user['认证'][user['雪球官方账号认证']==True]=user['认证'][user['雪球官方账号认证']==True].apply(lambda x:x+['雪球官方账号认证'])
user['认证']=user['认证'].apply(lambda x:','.join(x))

/Users/priscilla/Desktop/python/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/priscilla/Desktop/python/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/priscilla/Desktop/python/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/

In [44]:
# influence
influ=pd.read_table('/Users/priscilla/Desktop/python/Fuguo/xueqiu_raw/part-00000',header=None)
influ['user_id']=influ[0].apply(lambda x:int(re.split(',|\(|\)',x)[1]))
influ['influence']=influ[0].apply(lambda x:float(re.split(',|\(|\)',x)[2]))
influ.set_index('user_id',inplace=True)


In [45]:
# tweet: only consider the last half year
tweet=pd.read_excel('/Users/priscilla/Desktop/python/Fuguo/xueqiu_result/tweet_simple.xlsx')
tweet=tweet[['user_id','created_at','retweet_status_id']]

# select the data after 2019-07-01
tweet=tweet[tweet['created_at']>='2019-07-01 00:00:00']

tweet['original']=(tweet['retweet_status_id']==0)+0
tweet['retweet']=(tweet['retweet_status_id']!=0)+0
tweet['count_tweet']=1
grouped=tweet.groupby('user_id')
tweet=grouped.sum()
tweet['original_per']=tweet['original']/tweet['count_tweet']
tweet['retweet_per']=tweet['retweet']/tweet['count_tweet']


In [46]:
# select columns
stock_group=stock_group[['count_stock','A','H','US']]
portfolio=portfolio[['count_portfolio','exist_period','annualized_gain','monthly_gain','1y_return','3y_return','best_port']]
article=article[['count_article','view_count','title']]
tweet=tweet[['count_tweet','original_per']]
influ=influ[['influence']]
user=user[['screen_name','verified_description','description','followers_count','friends_count','认证']]


In [47]:
#user_influ
user_influ=pd.merge(user,influ,left_index=True,right_index=True,how='left')
user_influ.columns=['昵称','认证介绍','个人简介','粉丝数','关注数','认证','影响力']
user_influ.sort_values(by='影响力',ascending=False,inplace=True)
#user_influ.to_excel('/Users/priscilla/Desktop/python/Fuguo/xueqiu_result/output_table/user_influ.xlsx')

In [48]:
name_id=user_influ[['昵称']]

In [49]:
#article_tweet
article_tweet=pd.merge(name_id,article,left_index=True,right_index=True,how='left')
article_tweet=pd.merge(article_tweet,tweet,left_index=True,right_index=True,how='left')
article_tweet.columns=['昵称','发文数','平均文章阅读数','代表文章','发帖数（近半年）','原创比率']
#article_tweet.to_excel('/Users/priscilla/Desktop/python/Fuguo/xueqiu_result/output_table/article_tweet.xlsx')

In [50]:
#region_indu
region_indu=pd.merge(name_id,stock_group[['A','H','US','count_stock']],left_index=True,right_index=True,how='left')
region_indu=pd.merge(region_indu,indu,left_index=True,right_index=True,how='left')
region_indu.columns=['昵称','中国大陆','香港','美国','关注股票数','关注行业']
#region_indu.to_excel('/Users/priscilla/Desktop/python/Fuguo/xueqiu_result/output_table/region_indu.xlsx')

In [53]:
# portfolio
port=pd.merge(name_id,portfolio,left_index=True,right_index=True,how='left')
port.columns=['昵称','创建组数','组合平均持续时间','年化总收益率','月收益率','近1年年化收益率','近3年年化收益率','代表型组合']
port.to_excel('/Users/priscilla/Desktop/python/Fuguo/xueqiu_result/output_table/portfolio.xlsx')

In [54]:
# aggregate all
result=pd.merge(user_influ,article_tweet.iloc[:,1:],left_index=True,right_index=True,how='left')
result=pd.merge(result,region_indu.iloc[:,1:],left_index=True,right_index=True,how='left')
result=pd.merge(result,port.iloc[:,1:],left_index=True,right_index=True,how='left')
result.to_excel('/Users/priscilla/Desktop/python/Fuguo/xueqiu_result/output_table/aggregated_table_new.xlsx')